In [1]:
import os
import json

import numpy as np
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt

import scanpy as sc

import statsmodels.stats.multitest

In [2]:
def get_confusion_count_df(df, col1, col2):
    '''Get the confusion matrix between two categorical columns in a dataframe.'''
    assert((col1 != 'count') and (col2 != 'count'))
    count_df = df[[col1, col2]].copy()
    count_df['count'] = 1
    
    conf_df = pd.pivot_table(count_df, index=[col1], columns=[col2], 
                              values='count', aggfunc=np.sum).fillna(0)
    
    return conf_df
        
def get_expected_count_df(conf_df):
    mtx = conf_df.values
    
    total_count = np.sum(mtx)
    row_fractions = np.sum(mtx, axis=1) / total_count
    col_fractions = np.sum(mtx, axis=0) / total_count
    
    expect_fractions = row_fractions[:, np.newaxis] * col_fractions[np.newaxis, :]
    expected_count_df = pd.DataFrame(data=expect_fractions * total_count, 
                                     index=conf_df.index, columns=conf_df.columns)
    
    return expected_count_df

In [3]:
output_path = 'cells_by_regions'
os.makedirs(output_path, exist_ok=True)

In [4]:
# Load the cell type labels
adata = sc.read_h5ad('/home/xingjiepan/data/whole_brain/analysis/20230723_final_integration/integration_workspace/adata_merfish_label_transfer.h5ad')
df_ct_labels = adata.obs.copy()
df_ct_labels = df_ct_labels[df_ct_labels['adjusted_subclass_label_confidence'] > 0.8]
df_ct_labels = df_ct_labels[df_ct_labels['subclass_label_transfer'] != 'LQ'].copy()

In [5]:
adata

AnnData object with n_obs × n_vars = 9343481 × 1122
    obs: 'integration_partition_transfer', 'integration_partition_confidence', 'subclass_label_transfer', 'subclass_label_confidence', 'cl_transfer', 'cl_confidence', 'cluster_cosine_similarity', 'adjusted_subclass_label_confidence', 'adjusted_cl_confidence'

In [6]:
registration_file = '/home/xingjiepan/data/whole_brain/CCF_registration/20230801/wb3_cell_metadata_cp2_integration_sm1_ccf_v1a.csv'
registration_df = pd.read_csv(registration_file, index_col=0, low_memory=False)

In [7]:
registration_df.columns

Index(['fov', 'volume', 'center_x', 'center_y', 'sample_id', 'slice_id',
       'fov_x', 'fov_y', 'integration_partition_transfer',
       'integration_partition_confidence', 'subclass_label_transfer',
       'subclass_label_confidence', 'cl_transfer', 'cl_confidence',
       'cluster_cosine_similarity', 'adjusted_subclass_label_confidence',
       'adjusted_cl_confidence', 'spatial_modules_level_1', 'sm1_color',
       'local_color', 'spatial_modules_level_1_name', 'sm1n_color', 'ccfx',
       'ccfy', 'ccfz', 'annotation', 'ccfx_2', 'ccfy_2', 'ccfz_2',
       'annotation_2', 'fixed_x', 'fixed_y'],
      dtype='object')

In [8]:
df_ct_labels['center_x'] = registration_df['center_x']
df_ct_labels['center_y'] = registration_df['center_y']
df_ct_labels['sample_id'] = registration_df['sample_id']
df_ct_labels['slice_id'] = registration_df['slice_id']
df_ct_labels['annotation_2'] = registration_df['annotation_2']
df_ct_labels['ccfx_2'] = registration_df['ccfx_2']
df_ct_labels['ccfy_2'] = registration_df['ccfy_2']
df_ct_labels['ccfz_2'] = registration_df['ccfz_2']

In [9]:
# Get the major brain regions of each cell
with open('/home/xingjiepan/data/whole_brain/CCF_registration/dict_annot_to_region_pons_medulla.json') as f:
    anno_to_region_map = json.load(f)

anno_to_region_map = {float(k):anno_to_region_map[k] for k in anno_to_region_map}
df_ct_labels['major_region'] = df_ct_labels['annotation_2'].map(anno_to_region_map)
df_ct_labels

,integration_partition_transfer,integration_partition_confidence,subclass_label_transfer,subclass_label_confidence,cl_transfer,cl_confidence,cluster_cosine_similarity,adjusted_subclass_label_confidence,adjusted_cl_confidence,center_x,center_y,sample_id,slice_id,annotation_2,ccfx_2,ccfy_2,ccfz_2,major_region
234935021349889588272390626853122325011,p14,0.999994,L4/5 IT CTX Glut,0.999995,175,0.574011,0.348292,0.999989,0.574007,-1881.294051,-3029.698505,co1_sample11,co1_slice35,484682528.0,6343.795425,6311.607800,3462.268750,Fiber_tracts
191312233909860241222434627810326092301,p14,0.923351,L4/5 IT CTX Glut,0.999989,188,0.539079,0.326353,0.923342,0.497760,-1184.186554,-1860.349754,co1_sample11,co1_slice35,131.0,6343.947450,5532.226800,2353.676175,Cortical_subplate
323524287250658343877175793322711768155,p14,0.882508,L4/5 IT CTX Glut,0.999986,182,0.185929,0.504082,0.882495,0.164083,-1153.860805,-1309.446003,co1_sample11,co1_slice35,961.0,6356.563225,5534.689250,1838.482800,Olfactory
167661822891342906039179419220149588845,p14,0.836117,L4/5 IT CTX Glut,0.999997,188,0.716450,0.339350,0.836114,0.599036,-936.560481,-1271.037004,co1_sample11,co1_slice35,344.0,6399.394700,5290.077300,1744.261825,Isocortex
170524074209935379967434977505767424561,p14,0.958276,L4/5 IT CTX Glut,0.999987,196,0.280663,0.401776,0.958264,0.268952,-1084.386301,-1231.977005,co1_sample11,co1_slice35,961.0,6371.982650,5459.941050,1742.863425,Olfactory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192023727430088544087887970184179370921,p9,0.999990,Oligo NN,0.999976,5231,0.815896,0.631906,0.999966,0.815888,-2470.281533,338.794500,sa2_sample1,sa2_slice1,657.0,4308.209950,3487.993575,1761.104475,Isocortex
337797600429886337272913887639112580963,p9,0.999989,Oligo NN,0.999996,5231,0.750674,0.554838,0.999986,0.750666,-2453.550033,410.080498,sa2_sample1,sa2_slice1,657.0,4318.529600,3578.220875,1756.962550,Isocortex
316103013919934052511280692479369243883,p9,0.999992,Oligo NN,0.999980,5231,0.984937,0.499332,0.999972,0.984929,-2645.157033,233.406494,sa2_sample1,sa2_slice1,878.0,4155.151975,3331.348800,1771.037050,Isocortex
61822534463540761959701154027601668848,p9,0.812865,Oligo NN,0.999997,5231,0.801477,0.066740,0.812862,0.651493,-2804.351530,402.177998,sa2_sample1,sa2_slice1,878.0,4001.432275,3515.032550,1768.554550,Isocortex


In [10]:
major_region_splitted = np.array(df_ct_labels['major_region'])

major_region_splitted[(df_ct_labels['major_region'] == 'Hypothalamus')
                     &(df_ct_labels['ccfx_2'] < 6481.4041375)] = 'anterior_HY'
major_region_splitted[(df_ct_labels['major_region'] == 'Hypothalamus')
                     &(df_ct_labels['ccfx_2'] >= 6481.4041375)] = 'posterior_HY'
major_region_splitted[(df_ct_labels['major_region'] == 'Midbrain')
                     &(df_ct_labels['ccfx_2'] < 9191.019475000001)] = 'anterior_MB'
major_region_splitted[(df_ct_labels['major_region'] == 'Midbrain')
                     &(df_ct_labels['ccfx_2'] >= 9191.019475000001)] = 'posterior_MB'

df_ct_labels['major_region_splitted'] = major_region_splitted
df_ct_labels['major_region_splitted'].astype('category').cat.categories

Index(['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB'],
      dtype='object')

In [11]:
regions_of_interest = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

df_ct_labels = df_ct_labels[df_ct_labels['major_region_splitted'].isin(regions_of_interest)]
df_ct_labels

,integration_partition_transfer,integration_partition_confidence,subclass_label_transfer,subclass_label_confidence,cl_transfer,cl_confidence,cluster_cosine_similarity,adjusted_subclass_label_confidence,adjusted_cl_confidence,center_x,center_y,sample_id,slice_id,annotation_2,ccfx_2,ccfy_2,ccfz_2,major_region,major_region_splitted
234935021349889588272390626853122325011,p14,0.999994,L4/5 IT CTX Glut,0.999995,175,0.574011,0.348292,0.999989,0.574007,-1881.294051,-3029.698505,co1_sample11,co1_slice35,484682528.0,6343.795425,6311.607800,3462.268750,Fiber_tracts,Fiber_tracts
191312233909860241222434627810326092301,p14,0.923351,L4/5 IT CTX Glut,0.999989,188,0.539079,0.326353,0.923342,0.497760,-1184.186554,-1860.349754,co1_sample11,co1_slice35,131.0,6343.947450,5532.226800,2353.676175,Cortical_subplate,Cortical_subplate
323524287250658343877175793322711768155,p14,0.882508,L4/5 IT CTX Glut,0.999986,182,0.185929,0.504082,0.882495,0.164083,-1153.860805,-1309.446003,co1_sample11,co1_slice35,961.0,6356.563225,5534.689250,1838.482800,Olfactory,Olfactory
167661822891342906039179419220149588845,p14,0.836117,L4/5 IT CTX Glut,0.999997,188,0.716450,0.339350,0.836114,0.599036,-936.560481,-1271.037004,co1_sample11,co1_slice35,344.0,6399.394700,5290.077300,1744.261825,Isocortex,Isocortex
170524074209935379967434977505767424561,p14,0.958276,L4/5 IT CTX Glut,0.999987,196,0.280663,0.401776,0.958264,0.268952,-1084.386301,-1231.977005,co1_sample11,co1_slice35,961.0,6371.982650,5459.941050,1742.863425,Olfactory,Olfactory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192023727430088544087887970184179370921,p9,0.999990,Oligo NN,0.999976,5231,0.815896,0.631906,0.999966,0.815888,-2470.281533,338.794500,sa2_sample1,sa2_slice1,657.0,4308.209950,3487.993575,1761.104475,Isocortex,Isocortex
337797600429886337272913887639112580963,p9,0.999989,Oligo NN,0.999996,5231,0.750674,0.554838,0.999986,0.750666,-2453.550033,410.080498,sa2_sample1,sa2_slice1,657.0,4318.529600,3578.220875,1756.962550,Isocortex,Isocortex
316103013919934052511280692479369243883,p9,0.999992,Oligo NN,0.999980,5231,0.984937,0.499332,0.999972,0.984929,-2645.157033,233.406494,sa2_sample1,sa2_slice1,878.0,4155.151975,3331.348800,1771.037050,Isocortex,Isocortex
61822534463540761959701154027601668848,p9,0.812865,Oligo NN,0.999997,5231,0.801477,0.066740,0.812862,0.651493,-2804.351530,402.177998,sa2_sample1,sa2_slice1,878.0,4001.432275,3515.032550,1768.554550,Isocortex,Isocortex


In [12]:
# Get the confusion matrix data frame
conf_df = get_confusion_count_df(df_ct_labels, 'subclass_label_transfer', 'major_region_splitted')

# Calculate the enrichment matrix data frame
expected_count_df = get_expected_count_df(conf_df)
region_enrichment_df = conf_df / expected_count_df.values
region_enrichment_df

major_region_splitted,Cerebellum,Cortical_subplate,Fiber_tracts,Hippocampus,Isocortex,Medulla,Olfactory,Pallidum,Pons,Striatum,Thalamus,Ventricular_systems,anterior_HY,anterior_MB,posterior_HY,posterior_MB
subclass_label_transfer,,,,,,,,,,,,,,,,
ABC NN,1.526826,0.173356,1.343101,1.612967,1.204255,0.734808,0.928931,0.067724,0.883401,0.130408,0.001917,1.649082,0.023397,0.082997,1.749435,1.143647
ACB-BST-FS D1 Gaba,0.000000,0.265188,0.173469,0.000000,0.000000,0.000000,0.033611,33.934838,0.000000,3.617796,0.000000,0.000000,4.458443,0.000000,0.000000,0.000000
AD Serpinb7 Glut,0.000000,0.000000,0.584259,0.000000,0.000000,0.000000,0.000000,0.000000,0.036784,0.000000,26.771438,0.915337,0.000000,0.000000,0.000000,0.031397
ADP-MPO Trp73 Glut,0.000000,0.000000,0.384473,0.000000,0.011671,0.000000,0.383986,8.839108,0.000000,0.816707,0.297801,0.341959,46.878108,0.000000,0.145926,0.000000
AHN Onecut3 Gaba,0.000000,0.000000,0.376859,0.006230,0.002092,0.013465,0.008096,0.285139,0.018060,0.039031,0.015249,0.000000,57.720372,0.048297,8.438415,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VLMC NN,1.355661,0.319049,1.327435,1.420367,0.846888,1.196320,0.810961,0.632748,0.921080,0.407049,0.703650,2.918563,0.333436,1.033858,1.413590,1.488698
VMH Fezf1 Glut,0.000000,0.000000,0.040451,0.000000,0.003760,0.000000,0.000000,0.000000,0.032471,0.010025,0.027416,0.073455,2.542536,0.000000,62.817326,0.000000
VMH Nr5a1 Glut,0.000000,0.000000,0.210390,0.000000,0.000718,0.004621,0.001389,0.000000,0.030988,0.005740,0.005233,0.056081,22.706178,0.011049,42.969064,0.000000


In [13]:
major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

fine_regions = ['anterior_HY', 'posterior_HY',
       'anterior_MB', 'posterior_MB', 'Pons', 'Medulla',]

for r in major_brain_regions:
    print(r)
        
    region_df_ct_labels = df_ct_labels[df_ct_labels['major_region_splitted'] == r]
    
    subclasses, counts = np.unique(region_df_ct_labels['subclass_label_transfer'], return_counts=True)
    selected_subclasses = []
    
    for i in np.argsort(-counts):
        
        selected = False
        # For non-neuronal, non-astrocytes
        if subclasses[i].endswith('NN') and (not subclasses[i].startswith('Astro')):       
            if counts[i] > 50:
                selected = True
                selected_subclasses.append(subclasses[i])
           
        # For astrocytes
        elif subclasses[i].startswith('Astro'):
            if region_enrichment_df.loc[subclasses[i], r] > 1:
                selected = True
                selected_subclasses.append(subclasses[i])
            
        # For neurons
        else:
            if r in fine_regions:
                threshold = 6
            else:
                threshold = 2
            
            if region_enrichment_df.loc[subclasses[i], r] > threshold:
                selected = True
                selected_subclasses.append(subclasses[i])
                
        print('\t', subclasses[i], counts[i], selected)
    
    region_df_ct_labels = region_df_ct_labels[region_df_ct_labels['subclass_label_transfer'].isin(
                                selected_subclasses)].copy()
    region_df_ct_labels.to_csv(os.path.join(output_path, f'{r}.csv'))
        

Cerebellum
	 CB Granule Glut 349943 True
	 CBX MLI Megf11 Gaba 80752 True
	 Endo NN 44536 True
	 Bergmann NN 35196 True
	 Oligo NN 29125 True
	 CBX MLI Cdh22 Gaba 19827 True
	 VLMC NN 11216 True
	 Astro-NT NN 9409 False
	 Peri NN 6695 True
	 CBX Purkinje Gaba 5817 True
	 OPC NN 5370 True
	 Microglia NN 4834 True
	 SMC NN 3678 True
	 CB PLI Gly-Gaba 2709 True
	 CHOR NN 2546 True
	 Astro-CB NN 2537 True
	 ABC NN 2217 True
	 Ependymal NN 1882 True
	 DCO UBC Glut 1713 True
	 BAM NN 1384 True
	 IC Tfap2d Maf Glut 1111 False
	 CBN Dmbx1 Gaba 1091 True
	 CBX Golgi Gly-Gaba 850 True
	 VCO Mafa Meis2 Glut 703 True
	 CBN Neurod2 Pvalb Glut 404 True
	 Lymphoid NN 403 True
	 DCO Il22 Gly-Gaba 373 True
	 PG-TRN-LRN Fat2 Glut 263 False
	 PB Lmx1a Glut 189 False
	 PB Evx2 Glut 142 False
	 CU-ECU-SPVI Foxb1 Glut 128 False
	 PAS-MV Ebf2 Gly-Gaba 127 False
	 IC Six3 En2 Gaba 127 False
	 Astroependymal NN 99 False
	 CT SUB Glut 88 False
	 DC NN 76 True
	 DTN-LDT-IPN Otp Pax3 Gaba 74 False
	 MEA-COA-BMA C

Hippocampus
	 Astro-TE NN 88219 True
	 DG Glut 75054 True
	 Endo NN 63293 True
	 CA1-ProS Glut 51318 True
	 Oligo NN 35864 True
	 Microglia NN 23701 True
	 CA3 Glut 22184 True
	 L2/3 IT PPP Glut 21609 True
	 OPC NN 20719 True
	 L2/3 IT ENT Glut 18280 True
	 L6b/CT ENT Glut 14800 True
	 SUB-ProS Glut 11223 True
	 VLMC NN 10331 True
	 Sst Gaba 10113 True
	 Pvalb Gaba 8835 False
	 Peri NN 8714 True
	 L2/3 IT PIR-ENTl Glut 5385 False
	 NP SUB Glut 4727 True
	 L2 IT ENT-po Glut 4647 True
	 Vip Gaba 4424 False
	 SMC NN 4249 True
	 Lamp5 Lhx6 Gaba 3678 True
	 L2 IT PPP-APr Glut 3365 True
	 Lamp5 Gaba 3083 False
	 CT SUB Glut 2891 True
	 HPF CR Glut 2882 True
	 CA2-FC-IG Glut 2825 True
	 DG-PIR Ex IMN 2581 True
	 L5/6 IT TPE-ENT Glut 2202 True
	 ABC NN 2059 True
	 RHP-COA Ndnf Gaba 1873 True
	 Astro-NT NN 1854 False
	 Sncg Gaba 1817 True
	 BAM NN 1625 True
	 ENTmv-PA-COAp Glut 1122 True
	 L2/3 IT RSP Glut 1114 False
	 NP PPP Glut 1111 True
	 IT EP-CLA Glut 825 False
	 Pvalb chandelier Gaba 781

Olfactory
	 OB-in Frmd7 Gaba 300495 True
	 L2/3 IT PIR-ENTl Glut 62427 True
	 Endo NN 61515 True
	 OB-out Frmd7 Gaba 49865 True
	 Astro-TE NN 42229 False
	 IT AON-TT-DP Glut 30220 True
	 OB Eomes Ms4a15 Glut 27479 True
	 OB Trdn Gaba 26016 True
	 Oligo NN 25951 True
	 OB Dopa-Gaba 24199 True
	 Microglia NN 23218 True
	 OB Meis2 Thsd7b Gaba 22483 True
	 Astro-OLF NN 21296 True
	 OB-STR-CTX Inh IMN 21006 True
	 OEC NN 20616 True
	 OPC NN 17045 True
	 VLMC NN 9078 True
	 Peri NN 7975 True
	 COAp Grxcr2 Glut 5891 True
	 OB-mi Frmd7 Gaba 4607 True
	 Pvalb Gaba 4164 False
	 SMC NN 3220 True
	 Lamp5 Gaba 3143 False
	 Sst Gaba 2849 False
	 COAa-PAA-MEA Barhl2 Glut 2608 True
	 Vip Gaba 2338 False
	 IT EP-CLA Glut 1885 False
	 BAM NN 1832 True
	 ABC NN 1825 True
	 OT D3 Folh1 Gaba 1684 False
	 CLA-EPd-CTX Car3 Glut 1577 False
	 NDB-SI-ant Prdm12 Gaba 1324 True
	 L6 CT CTX Glut 1239 False
	 MEA-COA-BMA Ccdc42 Glut 1223 False
	 STR-PAL Chst9 Gaba 1209 False
	 Sncg Gaba 1044 False
	 Astro-NT NN 961

Pons
	 Oligo NN 72078 True
	 Astro-NT NN 31007 True
	 Endo NN 27133 True
	 PG-TRN-LRN Fat2 Glut 10065 True
	 OPC NN 6518 True
	 Microglia NN 5477 True
	 Peri NN 3830 True
	 PSV Pvalb Lhx2 Glut 3347 True
	 VLMC NN 2311 True
	 MB-MY Tph2 Glut-Sero 1947 True
	 CB Granule Glut 1817 False
	 SMC NN 1691 True
	 LDT-PCG Vsx2 Lhx4 Glut 1484 True
	 PB Evx2 Glut 1473 True
	 PB Lmx1a Glut 1442 True
	 NLL Gata3 Gly-Gaba 1425 True
	 DTN-LDT-IPN Otp Pax3 Gaba 1315 True
	 PRNc Otp Gly-Gaba 1129 True
	 MY Lhx1 Gly-Gaba 1080 True
	 CS-RPO Meis2 Gaba 1063 True
	 PCG-PRNr Vsx2 Nkx6-1 Glut 951 True
	 NLL-SOC Spp1 Glut 599 True
	 PRNc-NI-SG-RPO Vsx2 Nr4a2 Glut 556 True
	 CS-PRNr-PCG Tmem163 Otp Gaba 547 True
	 BAM NN 545 True
	 PRP-NI-PRNc-GRN Otp Glut 525 True
	 PB-NTS Phox2b Ebf3 Lmx1b Glut 508 True
	 NTS Dbh Glut 505 True
	 CS-PRNr-DR En1 Sox2 Gaba 450 True
	 CUN-PPN Evx2 Meis2 Glut 437 True
	 LDT-PCG-CS Gata3 Lhx1 Gaba 425 True
	 PDTg-PCG Pax6 Gaba 422 True
	 ABC NN 389 True
	 PAG-ND-PCG Onecut1 Gaba 37

Thalamus
	 Oligo NN 55922 True
	 TH Prkcd Grin2c Glut 45471 True
	 Astro-NT NN 31762 True
	 Endo NN 28782 True
	 RT-ZI Gnb3 Gaba 8244 True
	 MH Tac2 Glut 7597 True
	 Microglia NN 7591 True
	 OPC NN 7469 True
	 Peri NN 4806 True
	 PF Fzd5 Glut 4641 True
	 PVT-PT Ntrk1 Glut 4047 True
	 SPA-SPFm-SPFp-POL-PIL-PoT Sp9 Glut 2452 True
	 VLMC NN 2091 True
	 CM-IAD-CL-PCN Sema5b Glut 2034 True
	 LH Pou4f1 Sox1 Glut 1582 True
	 SMC NN 1570 True
	 RE-Xi Nox4 Glut 1328 True
	 AV Col27a1 Glut 1066 True
	 ZI Pax6 Gaba 1048 True
	 Ependymal NN 882 True
	 AD Serpinb7 Glut 862 True
	 LGv-SPFp-SPFm Nkx2-2 Tcf7l2 Gaba 767 True
	 LGv-ZI Otx2 Gaba 585 True
	 BAM NN 492 True
	 Astro-TE NN 379 False
	 SCsg Gabrr2 Gaba 294 False
	 Astroependymal NN 203 True
	 BST-po Iigp1 Glut 189 True
	 PRC-PAG Tcf7l2 Irx2 Glut 145 True
	 CHOR NN 119 True
	 Lymphoid NN 118 True
	 APN C1ql2 Glut 108 False
	 PH Pitx2 Glut 102 True
	 TRS-BAC Sln Glut 85 False
	 LHA Barhl2 Glut 84 True
	 PRT Tcf7l2 Gaba 80 False
	 PVH-SO-PVa Otp

anterior_HY
	 Astro-NT NN 17281 True
	 Oligo NN 14378 True
	 Endo NN 9302 True
	 OPC NN 3632 True
	 PVpo-VMPO-MPN Hmx2 Gaba 3528 True
	 AHN-RCH-LHA Otp Fezf1 Glut 3270 True
	 Microglia NN 3110 True
	 SCH Six6 Cdc14a Gaba 2931 True
	 PVH-SO-PVa Otp Glut 2489 True
	 MPO-ADP Lhx8 Gaba 1847 True
	 VMH Nr5a1 Glut 1778 True
	 SI-MPO-LPO Lhx8 Gaba 1744 True
	 AHN-SBPV-PVHd Pdrm12 Gaba 1675 True
	 AVPV-MEPO-SFO Tbr1 Glut 1664 True
	 LHA-AHN-PVH Otp Trh Glut 1590 True
	 NDB-SI-MA-STRv Lhx8 Gaba 1583 True
	 PVR Six3 Sox3 Gaba 1581 True
	 AHN Onecut3 Gaba 1551 True
	 Peri NN 1490 True
	 Ependymal NN 1308 True
	 SI-MA-LPO-LHA Skor1 Glut 1278 True
	 MPN-MPO-PVpo Hmx2 Glut 1269 True
	 BST-MPN Six3 Nrgn Gaba 1181 True
	 ADP-MPO Trp73 Glut 903 True
	 LHA Barhl2 Glut 743 True
	 SBPV-PVa Six6 Satb2 Gaba 644 True
	 MEA-BST Lhx6 Nfib Gaba 562 True
	 RT-ZI Gnb3 Gaba 550 False
	 ZI Pax6 Gaba 498 False
	 TU-ARH Otp Six6 Gaba 483 True
	 SMC NN 470 True
	 BST Tac2 Gaba 424 True
	 VLMC NN 406 True
	 MEA-BST Sox

posterior_HY
	 Oligo NN 18446 True
	 Astro-NT NN 17686 True
	 Endo NN 11199 True
	 MM Foxb1 Glut 3906 True
	 VMH Nr5a1 Glut 3591 True
	 ZI Pax6 Gaba 3543 True
	 OPC NN 3386 True
	 Microglia NN 2944 True
	 STN-PSTN Pitx2 Glut 2480 True
	 TU-ARH Otp Six6 Gaba 2376 True
	 MM-ant Foxb1 Glut 2201 True
	 PH-ant-LHA Otp Bsx Glut 2157 True
	 VLMC NN 1837 True
	 DMH Hmx2 Gaba 1826 True
	 Peri NN 1758 True
	 DMH-LHA Gsx1 Gaba 1592 True
	 PH-SUM Foxa1 Glut 1431 True
	 Tanycyte NN 1409 True
	 PMd-LHA Foxb1 Glut 1139 True
	 VMH Fezf1 Glut 1002 True
	 PH-LHA Foxb1 Glut 981 True
	 PMv-TMv Pitx2 Glut 964 True
	 SMC NN 852 True
	 DMH Nkx2-4 Glut 681 True
	 PH Pitx2 Glut 670 True
	 PVHd-DMH Lhx6 Gaba 668 True
	 ARH-PVp Tbx3 Glut 596 True
	 Ependymal NN 590 True
	 ARH-PVp Tbx3 Gaba 565 True
	 TMv-PMv Tbx3 Hist-Gaba 547 True
	 DMH Hmx2 Glut 475 True
	 TH Prkcd Grin2c Glut 433 False
	 DMH-LHA Vgll2 Glut 420 True
	 ABC NN 399 True
	 LHA Pmch Glut 351 True
	 TMd-DMH Foxd2 Gaba 321 True
	 BAM NN 303 True
	 AH